In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import ast
from datetime import datetime

### Calibrated data

In [ ]:
# Load the CSV files
gyro_df = pd.read_csv("2025-04-23-10-04-18-gyroscope_data.csv")
accel_df = pd.read_csv("2025-04-23-10-04-18-accelerometer_data.csv")

# Display the first few rows of each DataFrame
gyro_df.head()++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

FileNotFoundError: [Errno 2] No such file or directory: '2025-04-23-10-04-18-gyroscope_data.csv'

In [3]:
gyro_df.info(), accel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5832 entries, 0 to 5831
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   timestamp           5832 non-null   int64 
 1   timestamp_ms        5832 non-null   int64 
 2   sample_time_offset  5832 non-null   object
 3   calibrated_gyro_x   5832 non-null   object
 4   calibrated_gyro_y   5832 non-null   object
 5   calibrated_gyro_z   5832 non-null   object
dtypes: int64(2), object(4)
memory usage: 273.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5832 entries, 0 to 5831
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   timestamp           5832 non-null   int64 
 1   timestamp_ms        5832 non-null   int64 
 2   sample_time_offset  5832 non-null   object
 3   calibrated_accel_x  5832 non-null   object
 4   calibrated_accel_y  5832 non-null   object
 5   calibrated_

(None, None)

In [4]:
def expand_dataframe(df, list_columns):
    # Parse list-like string entries into actual lists
    for col in list_columns:
        df[col] = df[col].apply(ast.literal_eval)
    
    # Repeat each row based on the length of the lists in the first list column
    repeated_rows = []
    for _, row in df.iterrows():
        length = len(row[list_columns[0]])
        for i in range(length):
            new_row = {}
            for col in df.columns:
                if col in list_columns:
                    new_row[col] = row[col][i]
                else:
                    new_row[col] = row[col]
            repeated_rows.append(new_row)
    
    return pd.DataFrame(repeated_rows)


In [5]:
# Define the list-like columns for each DataFrame
gyro_list_columns = ['sample_time_offset', 'calibrated_gyro_x', 'calibrated_gyro_y', 'calibrated_gyro_z']
accel_list_columns = ['sample_time_offset', 'calibrated_accel_x', 'calibrated_accel_y', 'calibrated_accel_z']

# Expand both DataFrames
expanded_gyro_df = expand_dataframe(gyro_df, gyro_list_columns)
expanded_accel_df = expand_dataframe(accel_df, accel_list_columns)

# Display a sample of the expanded DataFrames
expanded_gyro_df.head(), expanded_accel_df.head()

(    timestamp  timestamp_ms  sample_time_offset  calibrated_gyro_x  \
 0  1115543429            22                   0          36.173965   
 1  1115543429            22                  10          32.394199   
 2  1115543429            22                  19          33.724197   
 3  1115543429            22                  29          32.988926   
 4  1115543429            22                  39          29.558924   
 
    calibrated_gyro_y  calibrated_gyro_z  
 0          12.218019          10.846073  
 1           3.643082          12.771422  
 2           7.248082          12.876422  
 3          13.548008           8.816011  
 4          10.923008           8.921011  ,
     timestamp  timestamp_ms  sample_time_offset  calibrated_accel_x  \
 0  1115543428           986                   0          203.913544   
 1  1115543428           986                  17          464.496704   
 2  1115543428           986                  27          478.058624   
 3  1115543428           

In [6]:
# Merge on both timestamp to align corresponding sensor readings
combined_df = pd.concat([
    expanded_accel_df,
    expanded_gyro_df], axis=1
)

# Display a sample of the merged DataFrame
combined_df.head()

,timestamp,timestamp_ms,sample_time_offset,calibrated_accel_x,calibrated_accel_y,calibrated_accel_z,timestamp,timestamp_ms,sample_time_offset,calibrated_gyro_x,calibrated_gyro_y,calibrated_gyro_z
0,1115543428,986,0,203.913544,-184.350006,-968.457397,1115543429,22,0,36.173965,12.218019,10.846073
1,1115543428,986,17,464.496704,-314.445404,-436.175476,1115543429,22,10,32.394199,3.643082,12.771422
2,1115543428,986,27,478.058624,-337.573456,-388.861511,1115543429,22,19,33.724197,7.248082,12.876422
3,1115543428,986,36,462.559296,-331.791473,-403.647125,1115543429,22,29,32.988926,13.548008,8.816011
4,1115543428,986,46,463.527985,-322.154755,-347.461823,1115543429,22,39,29.558924,10.923008,8.921011


In [7]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145800 entries, 0 to 145799
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   timestamp           145800 non-null  int64  
 1   timestamp_ms        145800 non-null  int64  
 2   sample_time_offset  145800 non-null  int64  
 3   calibrated_accel_x  145800 non-null  float64
 4   calibrated_accel_y  145800 non-null  float64
 5   calibrated_accel_z  145800 non-null  float64
 6   timestamp           145800 non-null  int64  
 7   timestamp_ms        145800 non-null  int64  
 8   sample_time_offset  145800 non-null  int64  
 9   calibrated_gyro_x   145800 non-null  float64
 10  calibrated_gyro_y   145800 non-null  float64
 11  calibrated_gyro_z   145800 non-null  float64
dtypes: float64(6), int64(6)
memory usage: 13.3 MB


In [8]:
combine_df_cleaned = combined_df.dropna()

In [9]:
combine_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145800 entries, 0 to 145799
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   timestamp           145800 non-null  int64  
 1   timestamp_ms        145800 non-null  int64  
 2   sample_time_offset  145800 non-null  int64  
 3   calibrated_accel_x  145800 non-null  float64
 4   calibrated_accel_y  145800 non-null  float64
 5   calibrated_accel_z  145800 non-null  float64
 6   timestamp           145800 non-null  int64  
 7   timestamp_ms        145800 non-null  int64  
 8   sample_time_offset  145800 non-null  int64  
 9   calibrated_gyro_x   145800 non-null  float64
 10  calibrated_gyro_y   145800 non-null  float64
 11  calibrated_gyro_z   145800 non-null  float64
dtypes: float64(6), int64(6)
memory usage: 13.3 MB


In [10]:
# Conversion factor from milli-G to m/s²
conversion_factor = 0.00981

# Apply the conversion
for col in ['calibrated_accel_x', 'calibrated_accel_y', 'calibrated_accel_z']:
    # Convert to float and apply the conversion factor
    combine_df_cleaned[col] = combine_df_cleaned[col].astype(float) * conversion_factor

In [11]:
combine_df_cleaned

,timestamp,timestamp_ms,sample_time_offset,calibrated_accel_x,calibrated_accel_y,calibrated_accel_z,timestamp,timestamp_ms,sample_time_offset,calibrated_gyro_x,calibrated_gyro_y,calibrated_gyro_z
0,1115543428,986,0,2.000392,-1.808474,-9.500567,1115543429,22,0,36.173965,12.218019,10.846073
1,1115543428,986,17,4.556713,-3.084709,-4.278881,1115543429,22,10,32.394199,3.643082,12.771422
2,1115543428,986,27,4.689755,-3.311596,-3.814731,1115543429,22,19,33.724197,7.248082,12.876422
3,1115543428,986,36,4.537707,-3.254874,-3.959778,1115543429,22,29,32.988926,13.548008,8.816011
4,1115543428,986,46,4.547210,-3.160338,-3.408600,1115543429,22,39,29.558924,10.923008,8.921011
...,...,...,...,...,...,...,...,...,...,...,...,...
145795,1115544886,744,199,0.565431,-1.099453,-9.655284,1115544886,772,200,2.291269,-4.470961,-2.363162
145796,1115544886,744,208,0.299346,-0.853660,-9.616605,1115544886,772,209,3.900143,-5.766175,-3.972761
145797,1115544886,744,218,0.384873,-1.033278,-9.674623,1115544886,772,219,4.985143,-6.956175,-5.477761
145798,1115544886,744,227,0.555928,-1.033278,-9.674623,1115544886,772,238,4.076911,-8.600839,-0.088391


In [15]:
print(combine_df_cleaned.columns)

Index(['timestamp_accel', 'timestamp_ms_accel', 'sample_time_offset_accel',
       'calibrated_accel_x', 'calibrated_accel_y', 'calibrated_accel_z',
       'timestamp_gyro', 'timestamp_ms_gyro', 'sample_time_offset_gyro',
       'calibrated_gyro_x', 'calibrated_gyro_y', 'calibrated_gyro_z',
       'time_ms_accel', 'time_ms_gyro'],
      dtype='object')


In [13]:
combine_df_cleaned.columns.values[0] = "timestamp_accel"
combine_df_cleaned.columns.values[1] = "timestamp_ms_accel"
combine_df_cleaned.columns.values[2] = "sample_time_offset_accel"
combine_df_cleaned.columns.values[6] = "timestamp_gyro"
combine_df_cleaned.columns.values[7] = "timestamp_ms_gyro"
combine_df_cleaned.columns.values[8] = "sample_time_offset_gyro"

In [14]:
combine_df_cleaned['time_ms_accel'] = combine_df_cleaned['timestamp_ms_accel'] + combine_df_cleaned['sample_time_offset_accel'].astype(int)
combine_df_cleaned['time_ms_gyro'] = combine_df_cleaned['timestamp_ms_gyro'] + combine_df_cleaned['sample_time_offset_gyro'].astype(int)

In [16]:
main_df = combine_df_cleaned.drop(columns=['sample_time_offset_accel', 'sample_time_offset_gyro', 'timestamp_ms_accel', 'timestamp_ms_gyro'])

In [17]:
main_df

,timestamp_accel,calibrated_accel_x,calibrated_accel_y,calibrated_accel_z,timestamp_gyro,calibrated_gyro_x,calibrated_gyro_y,calibrated_gyro_z,time_ms_accel,time_ms_gyro
0,1115543428,2.000392,-1.808474,-9.500567,1115543429,36.173965,12.218019,10.846073,986,22
1,1115543428,4.556713,-3.084709,-4.278881,1115543429,32.394199,3.643082,12.771422,1003,32
2,1115543428,4.689755,-3.311596,-3.814731,1115543429,33.724197,7.248082,12.876422,1013,41
3,1115543428,4.537707,-3.254874,-3.959778,1115543429,32.988926,13.548008,8.816011,1022,51
4,1115543428,4.547210,-3.160338,-3.408600,1115543429,29.558924,10.923008,8.921011,1032,61
...,...,...,...,...,...,...,...,...,...,...
145795,1115544886,0.565431,-1.099453,-9.655284,1115544886,2.291269,-4.470961,-2.363162,943,972
145796,1115544886,0.299346,-0.853660,-9.616605,1115544886,3.900143,-5.766175,-3.972761,952,981
145797,1115544886,0.384873,-1.033278,-9.674623,1115544886,4.985143,-6.956175,-5.477761,962,991
145798,1115544886,0.555928,-1.033278,-9.674623,1115544886,4.076911,-8.600839,-0.088391,971,1010


In [18]:
# Get the current column names as a list
cols = main_df.columns.tolist()
print(f"Original column order: {cols}")


Original column order: ['timestamp_accel', 'calibrated_accel_x', 'calibrated_accel_y', 'calibrated_accel_z', 'timestamp_gyro', 'calibrated_gyro_x', 'calibrated_gyro_y', 'calibrated_gyro_z', 'time_ms_accel', 'time_ms_gyro']


In [19]:

# Identify the column to move and its current position
column_to_move = 'time_ms_accel'
current_index = cols.index(column_to_move)

# Remove the column from its current position
cols.pop(current_index)

# Insert the column at the desired new position
new_index = 1
cols.insert(new_index, column_to_move)
print(f"New column order: {cols}")

# Reindex the DataFrame with the new column order
df_reordered = main_df[cols]
print(df_reordered)

New column order: ['timestamp_accel', 'time_ms_accel', 'calibrated_accel_x', 'calibrated_accel_y', 'calibrated_accel_z', 'timestamp_gyro', 'calibrated_gyro_x', 'calibrated_gyro_y', 'calibrated_gyro_z', 'time_ms_gyro']
        timestamp_accel  time_ms_accel  calibrated_accel_x  \
0            1115543428            986            2.000392   
1            1115543428           1003            4.556713   
2            1115543428           1013            4.689755   
3            1115543428           1022            4.537707   
4            1115543428           1032            4.547210   
...                 ...            ...                 ...   
145795       1115544886            943            0.565431   
145796       1115544886            952            0.299346   
145797       1115544886            962            0.384873   
145798       1115544886            971            0.555928   
145799       1115544886            981            0.860026   

        calibrated_accel_y  calibrate

In [20]:
# Identify the column to move and its current position
column_to_move = 'time_ms_gyro'
current_index = cols.index(column_to_move)

# Remove the column from its current position
cols.pop(current_index)

# Insert the column at the desired new position
new_index = 6
cols.insert(new_index, column_to_move)
print(f"New column order: {cols}")

# Reindex the DataFrame with the new column order
df_reordered = main_df[cols]
print(df_reordered)

New column order: ['timestamp_accel', 'time_ms_accel', 'calibrated_accel_x', 'calibrated_accel_y', 'calibrated_accel_z', 'timestamp_gyro', 'time_ms_gyro', 'calibrated_gyro_x', 'calibrated_gyro_y', 'calibrated_gyro_z']
        timestamp_accel  time_ms_accel  calibrated_accel_x  \
0            1115543428            986            2.000392   
1            1115543428           1003            4.556713   
2            1115543428           1013            4.689755   
3            1115543428           1022            4.537707   
4            1115543428           1032            4.547210   
...                 ...            ...                 ...   
145795       1115544886            943            0.565431   
145796       1115544886            952            0.299346   
145797       1115544886            962            0.384873   
145798       1115544886            971            0.555928   
145799       1115544886            981            0.860026   

        calibrated_accel_y  calibrate

In [21]:
df_reordered.head(20)

,timestamp_accel,time_ms_accel,calibrated_accel_x,calibrated_accel_y,calibrated_accel_z,timestamp_gyro,time_ms_gyro,calibrated_gyro_x,calibrated_gyro_y,calibrated_gyro_z
0,1115543428,986,2.000392,-1.808474,-9.500567,1115543429,22,36.173965,12.218019,10.846073
1,1115543428,1003,4.556713,-3.084709,-4.278881,1115543429,32,32.394199,3.643082,12.771422
2,1115543428,1013,4.689755,-3.311596,-3.814731,1115543429,41,33.724197,7.248082,12.876422
3,1115543428,1022,4.537707,-3.254874,-3.959778,1115543429,51,32.988926,13.548008,8.816011
4,1115543428,1032,4.547210,-3.160338,-3.408600,1115543429,61,29.558924,10.923008,8.921011
5,1115543428,1041,4.566216,-3.075256,-3.147516,1115543429,80,20.564034,3.993038,7.486176
6,1115543428,1051,4.499694,-2.886184,-2.954120,1115543429,89,14.788965,2.838020,6.296073
7,1115543428,1061,4.518700,-3.018534,-2.654357,1115543429,99,8.943966,-0.171980,7.381073
8,1115543428,1070,4.452179,-3.160338,-2.190207,1115543429,109,3.903965,-1.921980,7.416073
9,1115543428,1080,4.404664,-3.046895,-1.861434,1115543429,118,-1.626035,-3.146981,6.191073


In [22]:
print(df_reordered.columns)

Index(['timestamp_accel', 'time_ms_accel', 'calibrated_accel_x',
       'calibrated_accel_y', 'calibrated_accel_z', 'timestamp_gyro',
       'time_ms_gyro', 'calibrated_gyro_x', 'calibrated_gyro_y',
       'calibrated_gyro_z'],
      dtype='object')


In [23]:
df_reordered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145800 entries, 0 to 145799
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   timestamp_accel     145800 non-null  int64  
 1   time_ms_accel       145800 non-null  int64  
 2   calibrated_accel_x  145800 non-null  float64
 3   calibrated_accel_y  145800 non-null  float64
 4   calibrated_accel_z  145800 non-null  float64
 5   timestamp_gyro      145800 non-null  int64  
 6   time_ms_gyro        145800 non-null  int64  
 7   calibrated_gyro_x   145800 non-null  float64
 8   calibrated_gyro_y   145800 non-null  float64
 9   calibrated_gyro_z   145800 non-null  float64
dtypes: float64(6), int64(4)
memory usage: 11.1 MB


In [24]:
# Parse Timestamps
df_reordered['timestamp_accel'] = pd.to_datetime(df_reordered['timestamp_accel'], unit='s', origin='unix')
df_reordered['timestamp_gyro'] = pd.to_datetime(df_reordered['timestamp_gyro'], unit='s', origin='unix')

# Combine Timestamp and Offset (in seconds)
df_reordered['precise_timestamp_accel'] = df_reordered['timestamp_accel'] + pd.to_timedelta(df_reordered['time_ms_accel'], unit='ms')
df_reordered['precise_timestamp_gyro'] = df_reordered['timestamp_gyro'] + pd.to_timedelta(df_reordered['time_ms_gyro'], unit='ms')

# Create separate DataFrames for accelerometer and gyroscope data
accel_cols = ['precise_timestamp_accel', 'calibrated_accel_x', 'calibrated_accel_y', 'calibrated_accel_z']
gyro_cols = ['precise_timestamp_gyro', 'calibrated_gyro_x', 'calibrated_gyro_y', 'calibrated_gyro_z']

df_accel = df_reordered[accel_cols].copy()
df_gyro = df_reordered[gyro_cols].copy()

# Rename the precise timestamp columns for merging
df_accel = df_accel.rename(columns={'precise_timestamp_accel': 'merged_timestamp'})
df_gyro = df_gyro.rename(columns={'precise_timestamp_gyro': 'merged_timestamp'})

In [ ]:

# Ensure both DataFrames are sorted by the 'merged_timestamp' column
df_accel = df_accel.sort_values(by='merged_timestamp')
df_gyro = df_gyro.sort_values(by='merged_timestamp')

# Merge DataFrames based on the precise timestamp with a tolerance
df_merged = pd.merge_asof(
    df_accel,
    df_gyro,
    on='merged_timestamp',
    tolerance=pd.Timedelta('1ms'),
    direction='nearest'  #find the closest match within the tolerance
)
print(df_merged.head())

         merged_timestamp  calibrated_accel_x  calibrated_accel_y  \
0 2005-05-08 09:10:28.986            2.000392           -1.808474   
1 2005-05-08 09:10:29.003            4.556713           -3.084709   
2 2005-05-08 09:10:29.013            4.689755           -3.311596   
3 2005-05-08 09:10:29.022            4.537707           -3.254874   
4 2005-05-08 09:10:29.032            4.547210           -3.160338   

   calibrated_accel_z  calibrated_gyro_x  calibrated_gyro_y  calibrated_gyro_z  
0           -9.500567                NaN                NaN                NaN  
1           -4.278881                NaN                NaN                NaN  
2           -3.814731                NaN                NaN                NaN  
3           -3.959778          36.173965          12.218019          10.846073  
4           -3.408600          32.394199           3.643082          12.771422  


In [26]:
df_merged.dropna(inplace=True)

In [27]:
df_merged

,merged_timestamp,calibrated_accel_x,calibrated_accel_y,calibrated_accel_z,calibrated_gyro_x,calibrated_gyro_y,calibrated_gyro_z
3,2005-05-08 09:10:29.022,4.537707,-3.254874,-3.959778,36.173965,12.218019,10.846073
4,2005-05-08 09:10:29.032,4.547210,-3.160338,-3.408600,32.394199,3.643082,12.771422
5,2005-05-08 09:10:29.041,4.566216,-3.075256,-3.147516,33.724197,7.248082,12.876422
6,2005-05-08 09:10:29.051,4.499694,-2.886184,-2.954120,32.988926,13.548008,8.816011
7,2005-05-08 09:10:29.061,4.518700,-3.018534,-2.654357,29.558924,10.923008,8.921011
...,...,...,...,...,...,...,...
145795,2005-05-08 09:34:46.943,0.565431,-1.099453,-9.655284,0.365947,-0.376022,0.961952
145796,2005-05-08 09:34:46.952,0.299346,-0.853660,-9.616605,1.135947,-1.391022,-0.298048
145797,2005-05-08 09:34:46.962,0.384873,-1.033278,-9.674623,1.661268,-2.825961,-0.823162
145798,2005-05-08 09:34:46.971,0.555928,-1.033278,-9.674623,2.291269,-4.470961,-2.363162


In [28]:
df_merged.columns

Index(['merged_timestamp', 'calibrated_accel_x', 'calibrated_accel_y',
       'calibrated_accel_z', 'calibrated_gyro_x', 'calibrated_gyro_y',
       'calibrated_gyro_z'],
      dtype='object')

In [29]:
def rename_columns_to_wrist(df, old_columns):
    """
    Renames specific columns in a dataframe to match the required wrist sensor column names
    
    Parameters:
    df (pandas.DataFrame): Input dataframe whose columns need to be renamed
    old_columns (list): List of current column names
        
    Returns:
    pandas.DataFrame: Dataframe with renamed columns
    """
    # Create a dictionary for renaming
    columns = [ "timestamp","Ax_w", "Ay_w", "Az_w","Gx_w", "Gy_w", "Gz_w"] 
    rename_dict = dict(zip(old_columns, columns))
    
    # Rename the columns
    df = df.rename(columns=rename_dict)
    
    return df

In [30]:
df_merged = rename_columns_to_wrist(df_merged,['merged_timestamp', 'calibrated_accel_x', 'calibrated_accel_y',
       'calibrated_accel_z', 'calibrated_gyro_x', 'calibrated_gyro_y',
       'calibrated_gyro_z'])

In [ ]:
#df_merged.to_csv("g_data.csv", index=False)

In [31]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 137966 entries, 3 to 145799
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   timestamp  137966 non-null  datetime64[ns]
 1   Ax_w       137966 non-null  float64       
 2   Ay_w       137966 non-null  float64       
 3   Az_w       137966 non-null  float64       
 4   Gx_w       137966 non-null  float64       
 5   Gy_w       137966 non-null  float64       
 6   Gz_w       137966 non-null  float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 8.4 MB


#### downsample

In [32]:
# Set timestamp as the index
df_merged.set_index('timestamp', inplace=True)

# Resample to 50 Hz → every 20 milliseconds (1000ms / 50 = 20ms)
df_downsampled = df_merged.resample('20ms').mean().dropna().reset_index()
df_downsampled

,timestamp,Ax_w,Ay_w,Az_w,Gx_w,Gy_w,Gz_w
0,2005-05-08 09:10:29.020,4.542458,-3.207606,-3.684189,34.284082,7.930551,11.808748
1,2005-05-08 09:10:29.040,4.532955,-2.980720,-3.050818,33.356562,10.398045,10.846217
2,2005-05-08 09:10:29.060,4.518700,-3.018534,-2.654357,29.558924,10.923008,8.921011
3,2005-05-08 09:10:29.080,4.322304,-3.034290,-1.664815,14.765655,2.219692,7.054440
4,2005-05-08 09:10:29.100,3.996033,-2.928725,-1.010493,1.138965,-2.534480,6.803573
...,...,...,...,...,...,...,...
72634,2005-05-08 09:34:46.900,1.458718,-1.657216,-9.365190,-4.045018,0.393795,27.912296
72635,2005-05-08 09:34:46.920,0.912293,-1.520139,-9.611770,-0.700427,1.006692,8.241551
72636,2005-05-08 09:34:46.940,0.432389,-0.976557,-9.635944,0.750947,-0.883522,0.331952
72637,2005-05-08 09:34:46.960,0.470401,-1.033278,-9.674623,1.976269,-3.648461,-1.593162


In [ ]:
#df_downsampled.to_csv("f_data_50hz.csv", index=False)

### Label

In [ ]:
# Define activity intervals in datetime format
# For user f
activity_ranges = [
    ("00:04:59", "00:06:58", "ear_rubbing"),
    ("00:07:01", "00:08:56", "forehead_rubbing"),
    ("00:09:00", "00:11:10", "hair_pulling"),
    ("00:11:12", "00:13:03", "hand_scratching"),
    ("00:13:14", "00:14:55", "hand_tapping"),
    ("00:15:05", "00:16:58", "knuckles_cracking"),
    ("00:17:05", "00:19:05", "nail_biting"),
    ("00:19:14", "00:21:03", "nape_rubbing"),
    ("00:21:11", "00:22:40", "smoking"),
    ("00:23:05", "00:23:10", "smoking"),
    ("00:23:31", "00:23:45", "sitting"),
    ("00:23:51", "00:24:03", "sitting"),
    ("00:24:05", "00:25:35", "sitting"),
    ("00:25:52", "00:27:24", "standing"),
]


In [33]:
# Define activity intervals in datetime format
# For user m
activity_ranges = [
    ("09:10:35", "09:12:36", "ear_rubbing"),
    ("09:12:41", "09:14:45", "forehead_rubbing"),
    ("09:14:50", "09:16:53", "hair_pulling"),
    ("09:17:02", "09:19:10", "hand_scratching"),
    ("09:19:18", "09:21:19", "hand_tapping"),
    ("09:21:31", "09:23:31", "knuckles_cracking"),
    ("09:23:36", "09:25:35", "nail_biting"),
    ("09:25:40", "09:27:43", "nape_rubbing"),
    ("09:27:55", "09:29:48", "sitting"),
    ("09:29:55", "09:32:38", "smoking"),
    ("09:32:54", "09:34:32", "standing"),
]


In [175]:
#for user g
activity_ranges = [
    ("20:36:16", "20:38:16", "ear_rubbing"),
    ("20:38:18", "20:40:15", "forehead_rubbing"),
    ("20:40:19", "20:42:14", "hair_pulling"),
    ("20:42:22", "20:44:11", "hand_scratching"),
    ("20:44:18", "20:46:13", "hand_tapping"),
    ("20:46:18", "20:48:22", "knuckles_cracking"),
    ("20:48:27", "20:51:10", "nail_biting"),
    ("20:51:19", "20:53:13", "nape_rubbing"),
    ("20:53:17", "20:55:14", "smoking"),
    ("20:55:24", "20:57:39", "sitting"),
    ("20:57:50", "21:00:13", "standing"),
]

In [34]:
# Convert time strings to datetime.time
activity_ranges = [
    (datetime.strptime(start, "%H:%M:%S").time(), datetime.strptime(end, "%H:%M:%S").time(), label)
    for start, end, label in activity_ranges
]


In [35]:
def label_time(t):
    for start, end, label in activity_ranges:
        if start <= t <= end:
            return label
    return "unknown"

# Convert timestamp to datetime and extract time part
df_downsampled['timestamp'] = pd.to_datetime(df_downsampled['timestamp'])
df_downsampled['time_only'] = df_downsampled['timestamp'].dt.time

# Apply labeling
df_downsampled['activity'] = df_downsampled['time_only'].apply(label_time)

# Remove 'unknown' labeled rows
df_labeled = df_downsampled[df_downsampled['activity'] != 'unknown'].copy()

# Drop helper column 'time_only'
df_labeled.drop(columns=['time_only'], inplace=True)

df_labeled.head()

,timestamp,Ax_w,Ay_w,Az_w,Gx_w,Gy_w,Gz_w,activity
299,2005-05-08 09:10:35.000,4.306466,-0.541691,-7.511814,-45.948027,-19.772069,53.848914,ear_rubbing
300,2005-05-08 09:10:35.020,3.886747,-1.430329,-5.821213,-106.871227,-26.772032,41.628284,ear_rubbing
301,2005-05-08 09:10:35.040,3.601656,-0.291171,-5.531119,-146.596313,-31.409555,30.848150,ear_rubbing
302,2005-05-08 09:10:35.060,3.283304,1.344302,-6.043618,-169.993843,-33.894564,20.313112,ear_rubbing
303,2005-05-08 09:10:35.080,3.164516,3.036496,-6.353051,-150.498825,-33.982058,6.400642,ear_rubbing


In [36]:
df_labeled.to_csv("m_data_labeled.csv", index=False)